# Analysis of Stock Price and Daily Returns 
### Project Description and Objectives
This notebook presents teh quantitative analysis of a stock price change focusing on the following:<br>
1. Daily and annualised returns 
2. Historical Volatility  (Risk)
3. Annualised Voltatility (Risk) 

#### Author: Laurel Asimiea 


### 1. Analysis Setup 

In [23]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy.stats import norm
import datetime as dt

# Finance
import yfinance as yf

# Visuals using Bokeh
from bokeh.io import output_file,show,output_notebook,push_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool,CategoricalColorMapper,ColumnDataSource, HoverTool
from bokeh.palettes import Category20_11
from bokeh.layouts import row,column,gridplot

# Visuals using Plotly
import plotly.express as px
import plotly.graph_objects as go


# Set plotting backend of pandas
pd.set_option('plotting.backend', 'plotly')
# Bokeh settings throughout the notebook
output_notebook()
my_tools = "pan,box_zoom,undo,redo,xwheel_zoom,ywheel_zoom,hover,save,reset"


Loading BokehJS ...

### 2. Load Google Stock Data 

In [3]:
# The period of focus is the last 5 years. 
end_date = dt.datetime.now()
years_back = 5
start_date = end_date - dt.timedelta(days = 365*years_back)

# Stocks of interest - Bloom Energy ticker symbol = 'BE'
BE = yf.download('BE', start= start_date, end=end_date)



[*********************100%%**********************]  1 of 1 completed


### 3. Analyse Stock Price 

In [4]:
# A quick look at the stock data stocks.
BE  

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-06-26,11.500,12.150,11.484,11.82,11.82,1295800
2019-06-27,11.830,12.305,11.490,11.64,11.64,1165000
2019-06-28,11.710,12.400,11.710,12.27,12.27,8383200
2019-07-01,12.990,13.065,12.320,12.51,12.51,2297400
2019-07-02,12.550,12.740,11.950,12.63,12.63,2645300
...,...,...,...,...,...,...
2024-06-14,14.860,14.920,13.910,14.33,14.33,5936800
2024-06-17,14.105,14.250,13.580,13.95,13.95,5403000
2024-06-18,13.870,14.020,13.570,13.65,13.65,6893600


In [5]:
# A visual of the stock prices - Open Close and Adj_Close 
plot = figure(x_axis_type='datetime', width=800, height=300, tools=my_tools, title='Bloom Energy Stock Open and Close Prices')
plot.line(BE.index, BE['Close'],legend_label='Close',color='blue')
plot.line(BE.index, BE['Open'], legend_label='Open',color='green')
plot.line(BE.index, BE['Adj Close'], legend_label='Adj. Close',color='red')
plot.legend.click_policy = 'hide'
show(plot)

Observing the stock price in the plot above shows a major increasing trend from the start of 2020, exhibting a 413% increase by<br>
early 2021. The stock price then exhibits a downward trend, loosing 77% of its value by early 2024.
In the period between 2021 and 2024, 3 price 'spikes' are observed.    

In [6]:
# Calculating the stock return
BE_return = np.log(BE['Adj Close']/BE['Adj Close'].shift(1)).dropna()
BE_return

Date
2019-06-27   -0.015346
2019-06-28    0.052710
2019-07-01    0.019371
2019-07-02    0.009547
2019-07-03   -0.017572
                ...   
2024-06-14   -0.052340
2024-06-17   -0.026876
2024-06-18   -0.021740
2024-06-20    0.026032
2024-06-21   -0.088765
Name: Adj Close, Length: 1255, dtype: float64

In [27]:
# Option 1: Visualise stcok return using Plotly Express  
BE_return.plot(title="Bloom Energy Stock Prick returns", width=900, height=400)
             

In [7]:
# Option 2: Visualise stock return using Bokeh
close_series = BE_return #['Adj Close']

source = ColumnDataSource(data={'Date': close_series.index,'Adj_Close': close_series.values})

# Create a new plot with a title and axis labels
p = figure(title=" Bloom Energy Adj_Close Price over 5 years",
           x_axis_label='Date', y_axis_label='Price (USD)',
           x_axis_type='datetime', width=800, height=300)

# Add Close price line
p.line(x='Date', y='Adj_Close', source=source, legend_label='Adj Close', line_width=2, color='blue')

# Add tooltips
hover = HoverTool()
hover.tooltips = [("Date", "@Date{%F}"), ("Adj Close", "@Adj_Close{0.2f}")]
hover.formatters = {"@Date": "datetime"}
p.add_tools(hover)
show(p)

The adjusted close price shows a mean return close to 0 with pulsations or fluctuations higher than 0.2 before 2022. 

In [48]:
# Cumulative return 
BE_return_cumulative = (BE_return+1).cumprod()-1 
#BE_return_cumulative_log = BE_return.sum()
#BE_return_cumulative = np.exp(BE_return_cumulative_log) - 1
#BE_return_cumulative

# Visualise cumulative return
BE_return_cumulative.plot(title="Bloom Energy - Cumulative Stock Prick Return", width=800, height=400)

The cumulative return supports the trends analysis of the earlier Adj_Close price. Thus an investment of $1 in<br>
July 2019 would have depreciated in value return in June 2024 by about 87%. 

In [62]:
# A look at the historical volatility 

# Standard Deviation
BE_return_std_daily = BE_return.std(ddof=1)
BE_return_std_annual = BE_return_std_daily*np.sqrt(252)   # assuming there are 252 trading days in a year
print(f"The daily and annualised standard deviations of Bloom Energy stock return are {BE_return_std_daily} and {BE_return_std_annual} respectively")


The daily and annualised standard deviations of Bloom Energy stock return are 0.058435660036179185 and 0.9276373449218748 respectively


In [70]:
# Annualised Stock Return Summary

trading_days = 252 # 252 trading days per year
annual_risk_free_rate = 0.01 # 1% 
BE_annualised_return = BE_return.mean() * trading_days
BE_annualised_volatility = BE_return.std() * np.sqrt(trading_days)
BE_Sharpe_Ratio = BE_annualised_return - annual_risk_free_rate/BE_annualised_volatility

# presentation

print(f"Bloom Energy Annualised Return: {BE_annualised_return}")
print(f"Bloom Energy Annualised Volatility: {BE_annualised_volatility}")
print(f"Bloom Energy Annualised Sharpe Ratio: {BE_Sharpe_Ratio}")



Bloom Energy Annualised Return: 0.01630740021130104
Bloom Energy Annualised Volatility: 0.9276373449218748
Bloom Energy Annualised Sharpe Ratio: 0.005527325374143426


In [71]:
# Probability of stock a 30% stock return increase in 500days
BE_return_mean500 = 500 * BE_return.mean()
BE_return_std500 = 500 * BE_return.std()

prob_return30 = norm.cdf(0.3, BE_return_mean500, BE_return_std500)
print(f"The chance that Bloom Energy stock return shall have a 30% increase is {round(prob_return30*100,1)}%")

The chance that Bloom Energy stock return shall have a 30% increase is 50.4%


In [72]:
# Finally, the Value at Risk VaR using Variance method
VaR = norm.ppf(0.05, BE_return.mean(), BE_return.std())
print(f"There is a 5% chance that Bloom Energy stock daily return is worse than {round(VaR*100,1)}%")


There is a 5% chance that Bloom Energy stock daily return is worse than -9.6%
